In [ ]:
#code for three class siamese classification
import warnings


In [42]:
#important imports
import tensorflow as tf
import numpy as np
import shutil
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, Dropout, SeparableConv2D, Flatten, Dense, concatenate, MaxPooling2D
from keras.utils import plot_model
from tensorflow.keras import backend as K
import os
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
import random
from collections import deque
from tqdm.keras import TqdmCallback
from tensorflow.keras.regularizers import l1, l2, l1_l2
from collections import defaultdict

In [2]:
#original path for files

file_path_dilantin =  'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/dilantin'
file_path_keppra = 'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/keppra'
file_path_none = 'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/none_0.15'

In [12]:
def load_filenames(dilantin_path, keppra_path, none_path):
    dilantin_files = [os.path.join(dilantin_path, f) for f in os.listdir(dilantin_path)]
    keppra_files = [os.path.join(keppra_path, f) for f in os.listdir(keppra_path)]
    none_files = [os.path.join(none_path, f) for f in os.listdir(none_path)]

    data_files = dilantin_files + keppra_files + none_files
    labels = [0]*len(dilantin_files) + [1]*len(keppra_files) + [2]*len(none_files)

    data = list(zip(data_files, labels))

    np.random.shuffle(data)

    return data

In [16]:
data = load_filenames(file_path_dilantin, file_path_keppra, file_path_none)

In [14]:
file_paths , labels = zip(*data)
file_paths = list(file_paths)
labels = list(labels)

In [17]:
train_files, test_files, train_labels, test_labels = train_test_split(file_paths,labels, test_size = 0.4, random_state = 42 )

In [18]:
print(f"training set size: {len(train_files)}")
print(f'testing set size: {len(test_files)}')

training set size: 931
testing set size: 621


In [19]:
dilanin_no =0
keppra_no = 0
none_no = 0
def count_filenames(filenames):

    count_dict = {'dilantin': 0, 'none': 0, 'keppra': 0}
    
    for filename in filenames:
        for key in count_dict.keys():
            if filename.lower().find(key.lower())!= -1:
                count_dict[key] += 1
                
    return count_dict

result = count_filenames(train_files)
print("Counts:", result)

Counts: {'dilantin': 263, 'none': 339, 'keppra': 329}


In [46]:
def load_data(file_paths, labels):
    data_by_class = defaultdict(list)
    for file_path, label in zip(file_paths, labels):
        try:
            data = np.load(file_path, allow_pickle=True).reshape(-1, 19, 50, 1).astype('float32') / 255
            for sample in data:
                data_by_class[label].append(sample)
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    return data_by_class

# Create triplets from loaded data
def triplet_generator(data_by_class):
    while True:
        anchor_class = random.choice(list(data_by_class.keys()))
        positive_class = anchor_class
        negative_class = random.choice([cls for cls in data_by_class.keys() if cls != anchor_class])
        
        anchor = random.choice(data_by_class[anchor_class])
        positive = random.choice([x for x in data_by_class[positive_class] if not np.array_equal(x, anchor)])
        negative = random.choice(data_by_class[negative_class])
        
        yield (anchor, positive, negative), (0, 0, 0)  # Dummy labels

# Create a TensorFlow dataset from the triplet generator
def create_triplet_dataset(data_by_class, batch_size=32, buffer_size=10000):
    dataset = tf.data.Dataset.from_generator(
        lambda: triplet_generator(data_by_class),
        output_signature=(
            (tf.TensorSpec(shape=(19, 50, 1), dtype=tf.float32),
             tf.TensorSpec(shape=(19, 50, 1), dtype=tf.float32),
             tf.TensorSpec(shape=(19, 50, 1), dtype=tf.float32)),
            tf.TensorSpec(shape=(3,), dtype=tf.int32)
        )
    )
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [47]:
train_data = load_data(train_files, train_labels)
test_data= load_data(test_files, test_labels)

Error loading Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/dilantin/346_00003087_dilantin_montage_False.txt: No data left in file
Error loading Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/dilantin/.DS_Store: Failed to interpret file 'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/dilantin/.DS_Store' as a pickle
Error loading Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/none_0.15/.DS_Store: Failed to interpret file 'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/none_0.15/.DS_Store' as a pickle
Error loading Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/keppra/.DS_Store: Failed to interpret file 'Siezure med dataset/dataset_used_in_CBMS_paper/fft_images_100Hz_ws_1_overlap_0.1_1Hz_50Hz/keppra/.DS_Store' as a pickle


In [48]:
print

3

In [39]:
def triplet_loss(margin=1.0):
    def loss(y_true, y_pred):
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)

        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        basic_loss = pos_dist - neg_dist + margin
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
        return loss
    return loss

In [35]:
def create_base_network(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(63, activation='relu')(x)  
    return tf.keras.Model(inputs, x)

input_shape = (19, 50, 1)
base_network = create_base_network(input_shape)

In [36]:
anchor_input = tf.keras.Input(shape=input_shape, name="anchor")
positive_input = tf.keras.Input(shape=input_shape, name="positive")
negative_input = tf.keras.Input(shape=input_shape, name="negative")


anchor_embedding = base_network(anchor_input)
positive_embedding = base_network(positive_input)
negative_embedding = base_network(negative_input)


siamese_network = Model(inputs=[anchor_input, positive_input, negative_input],
                        outputs=[anchor_embedding, positive_embedding, negative_embedding])


In [43]:
siamese_network.compile(optimizer='adam', loss=triplet_loss())

In [44]:
siamese_network.fit(train_dataset, validation_data= validate_dataset ,epochs=10)

Epoch 1/10
   1761/Unknown 23s 12ms/step - loss: 0.0671

2024-06-07 11:52:58.955002: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: EOFError: No data left in file
Traceback (most recent call last):

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/var/folders/63/_xq94k6j5hg9kn00zf6sz9k80000gn/T/ipykernel_5555/4269163512.py", line 22, in data_generator_by_class
    positive_data = np.load(positive_file, allow_pickle=True).reshape(-1, 19, 50, 1).astype('float32') / 255
                    ^^^^

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
EOFError: No data left in file
Traceback (most recent call last):

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/homebrew/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/var/folders/63/_xq94k6j5hg9kn00zf6sz9k80000gn/T/ipykernel_5555/4269163512.py", line 22, in data_generator_by_class
    positive_data = np.load(positive_file, allow_pickle=True).reshape(-1, 19, 50, 1).astype(